The following code was tested only with Linux OS. The Oracle DB client was installed and the cx_Oracle python library was used with Python 3.5.

In [1]:
import cx_Oracle
import csv
import os

We set the language to UTF8 so we can import and read Unicode characters that exist in the dataset.

In [2]:
os.environ['NLS_LANG'] = '.AL32UTF8'
db = cx_Oracle.connect('user_1', 'babis', '52.50.54.24:1521/ORCL', encoding='utf-8')
print('Oracle DB server version %s' % db.version)

Oracle DB server version 12.1.0.2.0


Create the connection cursor we will use for all operations.

In [3]:
cursor = db.cursor()

In [6]:
for i, row in enumerate(cursor.execute('SELECT * FROM companies WHERE name LIKE \'Berk%\'')):
    print(row)
    if(i > 10):
        break

('/organization/berkeley-design-automation', 'Berkeley Design Automation', 'http://www.berkeley-da.com', 'Nanotechnology', 20250000, 'acquired', 'USA', 'CA', 'SF Bay Area', 'Santa Clara', 3, datetime.datetime(2001, 10, 1, 0, 0), datetime.datetime(2002, 12, 1, 0, 0), datetime.datetime(2007, 4, 1, 0, 0))
('/organization/berkeley-ultrasound', 'Berkeley Ultrasound', None, None, None, 'closed', None, None, None, None, 1, None, datetime.datetime(2015, 7, 14, 0, 0), datetime.datetime(2015, 7, 14, 0, 0))
('/organization/berkley-networks', 'Berkley Networks', 'http://inmarkit.com', 'E-Commerce', 1642500, 'operating', 'USA', 'NY', 'New York City', 'New York', 4, datetime.datetime(2005, 1, 1, 0, 0), datetime.datetime(2011, 1, 7, 0, 0), datetime.datetime(2014, 3, 27, 0, 0))
('/organization/berkna-wireless', 'Berkäna Wireless', 'http://www.berkanawireless.com', 'Mobile', 26500000, 'acquired', 'USA', 'CA', 'SF Bay Area', 'Campbell', 4, datetime.datetime(2001, 2, 1, 0, 0), datetime.datetime(2001, 10,

In [4]:
#cursor.execute('ALTER user user_1 IDENTIFIED BY "babis" REPLACE pwd')

We create the companies table. The length of the fields were acquired through trial and error. We also use the permalink column as Primary Key.

In [5]:
create_companies = ('CREATE TABLE companies ('
                    'permalink NVARCHAR2(120),'
                    'name NVARCHAR2(80),'
                    'homepage_url VARCHAR(350),'
                    'category_list VARCHAR(400),'
                    'funding_total_usd NUMBER(12,0),'
                    'status VARCHAR(15),'
                    'country_code CHAR(3),'
                    'state_code VARCHAR(2),'
                    'region VARCHAR(40),'
                    'city VARCHAR(40),'
                    'funding_rounds NUMBER(3,0),'
                    'founded_at DATE,'
                    'first_funding_at DATE,'
                    'last_funding_at DATE)')

add_pk_companies = ('ALTER TABLE companies ADD CONSTRAINT pk_permalink PRIMARY KEY (permalink)')

cursor.execute(create_companies)
cursor.execute(add_pk_companies)

We insert the data from file "companies.csv" in the companies table. Care is taken to tranform missing values in column "funding_total_usd" to NULL.

In [6]:
insert_companies = ('INSERT INTO companies (permalink, name, homepage_url, category_list, funding_total_usd, status, country_code, state_code, region, city, funding_rounds, founded_at, first_funding_at, last_funding_at) '
                    'VALUES ( :permalink, :name, :homepage_url, :category_list, :funding_total_usd, :status, :country_code, :state_code, :region, :city, :funding_rounds, TO_DATE(:founded_at, \'YYYY-MM-DD\'), TO_DATE(:first_funding_at, \'YYYY-MM-DD\'), TO_DATE(:last_funding_at, \'YYYY-MM-DD\'))')

with open('part1/companies.csv', encoding='utf-8') as companies_file:
    reader = csv.DictReader(companies_file)
    for row in reader:
        if row['funding_total_usd']=='-':
            row['funding_total_usd']=''
        cursor.execute(insert_companies, row)
    

We create the acquisitions table. Again column lengths was acquired through trial and error. In this dataset we have duplicate lines so we cannot use the permalink fields as Primary Key.

In [11]:
create_acquisitions = ('CREATE TABLE acquisitions ('
                       'company_permalink NVARCHAR2(120),'
                       'company_name NVARCHAR2(120),'
                       'company_category_list VARCHAR(400),'
                       'company_country_code CHAR(3),'
                       'company_state_code VARCHAR(2),'
                       'company_region VARCHAR(40),'
                       'company_city VARCHAR(40),'
                       'acquirer_permalink NVARCHAR2(120),'
                       'acquirer_name NVARCHAR2(120),'
                       'acquirer_category_list VARCHAR(400),'
                       'acquirer_country_code CHAR(3),'
                       'acquirer_state_code VARCHAR(2),'
                       'acquirer_region VARCHAR(40),'
                       'acquirer_city VARCHAR(40),'
                       'acquired_at DATE,'
                       'acquired_month DATE,'
                       'price_amount NUMBER(12,0),'
                       'price_currency_code CHAR(3))')

#add_pk_acquisitions = ('ALTER TABLE acquisitions ADD CONSTRAINT acquisitions_pk_permalink PRIMARY KEY (company_permalink, acquirer_permalink)')

cursor.execute(create_acquisitions)
#cursor.execute(add_pk_acquisitions)

We insert thte data from file "acquisitions.csv".

In [12]:
insert_acquisitions = ('INSERT INTO acquisitions (company_permalink, company_name, company_category_list, company_country_code, company_state_code, company_region, company_city, acquirer_permalink, acquirer_name, acquirer_category_list, acquirer_country_code, acquirer_state_code, acquirer_region, acquirer_city, acquired_at, acquired_month, price_amount, price_currency_code)'
                       'VALUES (:company_permalink, :company_name, :company_category_list, :company_country_code, :company_state_code, :company_region, :company_city, :acquirer_permalink, :acquirer_name, :acquirer_category_list, :acquirer_country_code, :acquirer_state_code, :acquirer_region, :acquirer_city, TO_DATE(:acquired_at, \'YYYY-MM-DD\'), TO_DATE(:acquired_month, \'YYYY-MM\'), :price_amount, :price_currency_code)')

with open('part1/acquisitions.csv', encoding='utf-8') as acquisitions_file:
    reader = csv.DictReader(acquisitions_file)
    for row in reader:
        cursor.execute(insert_acquisitions, row)

For all the companies whose status column is "acquired" we show the associated acquisition amount, or None if it doesn't exist. As the acquisitions data set is not clean we select distinct values only. We only print the first 10 entries to conserve space in the notebook.

In [ ]:
part1a = ('CREATE VIEW q1a AS '
          'SELECT DISTINCT acquisitions.company_name AS company_name, acquisitions.price_amount AS price_amount '
          'FROM acquisitions '
          'JOIN companies '
          'ON acquisitions.company_permalink = companies.permalink '
          'WHERE companies.status = \'acquired\' '
          'ORDER BY company_name')

cursor.execute(part1a)

In [17]:
for i, row in enumerate(cursor.execute('SELECT * FROM q1a')):
    print(row)
    if(i > 10):
        break

('1 Mainstream', None)
('1000 Markets', None)
('1000memories', None)
('100Plus', None)
('1010data', 500000000)
('12Society', None)
('13th Lab', None)
('170 Systems', 43000000)
('1CLICK', None)
('1DocWay', None)
('280 North', 20000000)
('29West', None)


According to these data 3064 startups where founded in 2012 and 2013.

In [20]:
part1b = ('CREATE VIEW q1b AS '
          'SELECT COUNT(name) AS count '
          'FROM companies '
          'WHERE founded_at >= TO_DATE(\'2012-01-01\', \'YYYY-MM-DD\') '
          'AND founded_at <= TO_DATE(\'2014-12-31\', \'YYYY-MM-DD\')')

cursor.execute(part1b)

In [21]:
for i, row in enumerate(cursor.execute('SELECT * FROM q1b')):
    print(row)
    if(i > 10):
        break

(4289,)


California is the state with the larger number of startups in the security market.

In [22]:
part1c = ('CREATE VIEW q1c AS '
          'SELECT a.* '
          'FROM '
              '(SELECT state_code AS state, COUNT(*) AS total '
              'FROM companies '
              'WHERE category_list LIKE \'%Security%\' '
              'AND state_code IS NOT NULL '
              'GROUP BY state_code) a '
          'JOIN '
              '(SELECT MAX(y.total) AS total '
              'FROM '
                  '(SELECT state_code AS state, COUNT(*) AS total '
                  'FROM companies '
                  'WHERE category_list LIKE \'%Security%\' '
                  'AND state_code IS NOT NULL '
                  'GROUP BY state_code) y '
              ') b '
          'ON a.total=b.total')

cursor.execute(part1c)

In [23]:
for i, row in enumerate(cursor.execute('SELECT * FROM q1c')):
    print(row)
    if(i > 10):
        break

('CA', 124)


The list with the cities that have more acquirer companies than startups is bellow. First 10 are print out to conserve notebook space.

In [24]:
part1d = ('CREATE VIEW q1d AS '
          'SELECT acquirers.city AS city '
          'FROM ('
              'SELECT city, COUNT(*) AS startups '
              'FROM companies '
              'WHERE founded_at >= TO_DATE(\'2012-01-01\', \'YYYY-MM-DD\') '
              'AND founded_at <= TO_DATE(\'2013-12-31\', \'YYYY-MM-DD\') '
              'AND city IS NOT NULL '
              'GROUP BY city) startups '
          'RIGHT JOIN ('
              'SELECT acquirer_city AS city, COUNT(DISTINCT acquirer_name) AS acquirers '
              'FROM acquisitions '
              'WHERE acquirer_city IS NOT NULL '
              'GROUP BY acquirer_city) acquirers '
          'ON startups.city=acquirers.city '
          'WHERE acquirers.acquirers > startups.startups'
         )

cursor.execute(part1d)

In [25]:
for i, row in enumerate(cursor.execute('SELECT * FROM q1d')):
    print(row)
    if(i > 10):
        break

('Tokyo',)
('London',)
('Bellevue',)
('Aliso Viejo',)
('Mississauga',)
('Salt Lake City',)
('Columbia',)
('Tulsa',)
('Newport Beach',)
('Bedford',)
('Avon',)
('Costa Mesa',)


We drop the tables and close the connection to the DB server.

In [ ]:
drop_companies = ('DROP TABLE companies')
cursor.execute(drop_companies)

In [ ]:
drop_acquisitions = ('DROP TABLE acquisitions')
cursor.execute(drop_acquisitions)

In [ ]:
cursor.close()
db.close()